In [37]:
import gurobipy as gp
import numpy as np
import matplotlib.pyplot as plt
import random
from random import randint
from sklearn.datasets import make_blobs
from utility import *

In [38]:
points = load_points(f'C:/Users/franc/Documents/GitHub/Ricerca_Operativa_2022/Ricerca_Operativa_2022/benchmark/benchmark{8}.txt')

K = 5
N = len(points)

dist = {}

for i in range(N):
    for j in range(N):
        dist[(i,j)] = np.sum((points[i]-points[j])**2)

In [39]:
model = gp.Model()
x = model.addVars(N,N, vtype = gp.GRB.BINARY, name = "X") #i elemento , j centroide
y = model.addVars(N, vtype = gp.GRB.BINARY, name = "Y")

obj = model.setObjective(x.prod(dist), gp.GRB.MINIMIZE)

model.addConstrs((sum(x[i,j] for j in range(N)) == 1 for i in range(N)))
model.addConstr(sum(y[j] for j in range(N))==K)
model.addConstrs((x[i,j]<=y[j] for i in range(N) for j in range(N)))

model.optimize()


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 490701 rows, 490700 columns and 1470700 nonzeros
Model fingerprint: 0x0fa9d914
Variable types: 0 continuous, 490700 integer (490700 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve time: 7.31s
Presolved: 490701 rows, 490700 columns, 1470700 nonzeros
Variable types: 0 continuous, 490700 integer (490700 binary)
Found heuristic solution: objective 148607.54987

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4675744e+05   0.000000e+00   1.981554e+08     12s
   80162    1.2214751e+05   0.000000e+00   5.724057e+08

In [40]:
#Converto la soluzione nel formato utilizzato dalle euristiche:

centroids = []
for j in range(N):
    if y[j].X == 1:
        centroids.append(j)

sol = np.zeros(N)

for i in range(N):
    for cluster,j in enumerate(centroids):
        if x[i,j].X == 1:
            sol[i] = cluster


print("{}".format(squared_inner_distance(sol, points, K)))

13695.301455840068
